In [1]:
# for fetching list of youtube videos
from youtubesearchpython import *
import pandas as pd

# for downloading audio and extracting text
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

# for saving embedding to vectordb
import os
import math
import pinecone
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

import openai
import requests

import unicodedata
import difflib # for fuzzy string matching

/Users/cueball/Documents/langchain/env/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# channels = ChannelsSearch('JacksonGalaxy')
# print(channels.result())
channel_id = "UCheL-cUqfzUB8dfM_rFOfDQ" #pull out id from channels above
playlist = Playlist(playlist_from_channel_id(channel_id))

while playlist.hasMoreVideos:
    print('Getting more videos...')
    playlist.getNextVideos()
    print(f'Videos retrieved: {len(playlist.videos)}')

Getting more videos...
Videos retrieved: 200
Getting more videos...
Videos retrieved: 300
Getting more videos...
Videos retrieved: 400
Getting more videos...
Videos retrieved: 500
Getting more videos...
Videos retrieved: 599
Getting more videos...
Videos retrieved: 699
Getting more videos...
Videos retrieved: 761


In [3]:
stor_metadata = pd.DataFrame()

# Excluding 'shorts' based on duration of videos
#from datetime import datetime, time
import time
time_format = '%H:%M:%S'
one_min = time.strptime('00:01:00', time_format)

count = 1
for v in playlist.videos:
    duration_str = v['duration']
    if len(duration_str) < 6:
        duration_str = '0:' + duration_str
    duration = time.strptime(duration_str, time_format)
    #print(duration_str, duration)
    if duration > one_min:
        #print(">>>LONG VIDEO")
        stor_metadata.loc[v['title'],'title']=v['title']
        stor_metadata.loc[v['title'],'link']=v['link']
        # using unicode data to fix issue with filenames
        # title_corrected = unicodedata.normalize('NFKD', v['title'])
        stor_metadata.loc[v['title'],'img']=v['thumbnails'][3]['url']
        stor_metadata.loc[v['title'],'id']=int(count)
        #print(count)
        count = count + 1        

In [4]:
#print(stor_metadata)


# Download, transcribe, and embed in batches to avoid connection errors
batch_size = 10
num_batches = math.ceil(len(stor_metadata) / batch_size)

batch_number = 23  # manually updated
# batch 13, numbers 120-130 skipped
# batch 17, nubers 160-170 skipped
# batch 19, numbers 180-190
# batch 22, numbers 210-220
# Set chunks to add
start_idx = (batch_number-1) * batch_size
end_idx = min(start_idx + batch_size, len(stor_metadata))
    
# Extract the current chunks
current_batch = stor_metadata[start_idx:end_idx]
print(start_idx)
print(end_idx)
stor_metadata[start_idx:end_idx]

220
230


,title,link,img,id
What is Catification and why it’s crucial for your cat-loving home!,What is Catification and why it’s crucial for ...,https://www.youtube.com/watch?v=4mi_l2w3a8s&li...,https://i.ytimg.com/vi/4mi_l2w3a8s/hqdefault.j...,222.0
Biting the Hand That Loves You: Aggression or Affection?,Biting the Hand That Loves You: Aggression or ...,https://www.youtube.com/watch?v=aYGdgsoRPOk&li...,https://i.ytimg.com/vi/aYGdgsoRPOk/hqdefault.j...,223.0
My Cat From Hell BIG NEWS - Live from the Cat Cave with Jackson Galaxy 5/23,My Cat From Hell BIG NEWS - Live from the Cat ...,https://www.youtube.com/watch?v=5PY-bihgNGY&li...,https://i.ytimg.com/vi/5PY-bihgNGY/hqdefault.j...,224.0
Tips on helping prevent Door-Dashing cats!,Tips on helping prevent Door-Dashing cats!,https://www.youtube.com/watch?v=i5odZTDV3So&li...,https://i.ytimg.com/vi/i5odZTDV3So/hqdefault.j...,225.0
Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips,Can my Cats Get Along? Cat-to-Cat Body Languag...,https://www.youtube.com/watch?v=UWohxDOXsl4&li...,https://i.ytimg.com/vi/UWohxDOXsl4/hqdefault.j...,226.0
What is the Flehmen Response AKA Stinkface?,What is the Flehmen Response AKA Stinkface?,https://www.youtube.com/watch?v=gW2QGYO5Ux0&li...,https://i.ytimg.com/vi/gW2QGYO5Ux0/hqdefault.j...,227.0
What does Separation Anxiety look like in your cat and how can you help?,What does Separation Anxiety look like in your...,https://www.youtube.com/watch?v=THhlirFJh6Y&li...,https://i.ytimg.com/vi/THhlirFJh6Y/hqdefault.j...,228.0
Live from the Cat Cave with Jackson Galaxy 5/9,Live from the Cat Cave with Jackson Galaxy 5/9,https://www.youtube.com/watch?v=_bw1jM87LUk&li...,https://i.ytimg.com/vi/_bw1jM87LUk/hqdefault.j...,229.0
All you need to know about cat allergies & what you can do about them!,All you need to know about cat allergies & wha...,https://www.youtube.com/watch?v=NKvuujrqnXY&li...,https://i.ytimg.com/vi/NKvuujrqnXY/hqdefault.j...,230.0
Cats that suckle: Is it Pica?,Cats that suckle: Is it Pica?,https://www.youtube.com/watch?v=jO52mm_pB5o&li...,https://i.ytimg.com/vi/jO52mm_pB5o/hqdefault.j...,231.0


In [5]:
urls=list(current_batch.link)
save_dir = "../Downloads/JacksonGalaxy_" + str(batch_number)
openai.api_key = os.environ["OPENAI_API_KEY"]
loader = GenericLoader(YoutubeAudioLoader(urls,save_dir),OpenAIWhisperParser())
docs = loader.load()

[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=4mi_l2w3a8s&list=UUheL-cUqfzUB8dfM_rFOfDQ&index=589&pp=iAQB
[youtube:tab] Downloading just the video 4mi_l2w3a8s because of --no-playlist
[youtube] Extracting URL: https://www.youtube.com/watch?v=4mi_l2w3a8s
[youtube] 4mi_l2w3a8s: Downloading webpage
[youtube] 4mi_l2w3a8s: Downloading ios player API JSON
[youtube] 4mi_l2w3a8s: Downloading android player API JSON
[youtube] 4mi_l2w3a8s: Downloading m3u8 information
[info] 4mi_l2w3a8s: Downloading 1 format(s): 140
[download] Destination: ../Downloads/JacksonGalaxy_23/What is Catification and why it’s crucial for your cat-loving home!.m4a
[download] 100% of    8.68MiB in 00:00:00 at 14.36MiB/s  
[FixupM4a] Correcting container of "../Downloads/JacksonGalaxy_23/What is Catification and why it’s crucial for your cat-loving home!.m4a"
[ExtractAudio] Not converting audio ../Downloads/JacksonGalaxy_23/What is Catification and why it’s crucial for your cat-loving home!.m4a; file is al

[youtube] Extracting URL: https://www.youtube.com/watch?v=NKvuujrqnXY
[youtube] NKvuujrqnXY: Downloading webpage
[youtube] NKvuujrqnXY: Downloading ios player API JSON
[youtube] NKvuujrqnXY: Downloading android player API JSON
[youtube] NKvuujrqnXY: Downloading m3u8 information
[info] NKvuujrqnXY: Downloading 1 format(s): 140
[download] Destination: ../Downloads/JacksonGalaxy_23/All you need to know about cat allergies & what you can do about them!.m4a
[download] 100% of   10.70MiB in 00:00:00 at 15.77MiB/s  
[FixupM4a] Correcting container of "../Downloads/JacksonGalaxy_23/All you need to know about cat allergies & what you can do about them!.m4a"
[ExtractAudio] Not converting audio ../Downloads/JacksonGalaxy_23/All you need to know about cat allergies & what you can do about them!.m4a; file is already in target format m4a
[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=jO52mm_pB5o&list=UUheL-cUqfzUB8dfM_rFOfDQ&index=598&pp=iAQB
[youtube:tab] Downloading just the video j

In [6]:
# Consolidate text per episode
concatenated_text_by_source = {}
for doc in docs:
    source = doc.metadata['source']
    page_content = doc.page_content
    if source in concatenated_text_by_source:
        concatenated_text_by_source[source] += ' ' + page_content
    else:
        concatenated_text_by_source[source] = page_content

In [7]:
# Split each video and add metadata we'll use in UI
splits=[]
metadatas=[]

# Split parameters
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 150)

# Build splits
for k in concatenated_text_by_source.keys():
    
    # Get downloaded title
    directory, filename = os.path.split(k)
    video_name = filename.split(".m4a")[0]    
    
    # Make splits
    episode_text = concatenated_text_by_source[k]
    episode_splits = text_splitter.split_text(episode_text)
    splits.append(episode_splits)
    
    # finding closest name instead of exact match to circumvent special character issues
    closest_name = difflib.get_close_matches(video_name, list(stor_metadata['title']))
    video_name_corrected = closest_name[0]
    print(video_name)
    print(closest_name)
    print(video_name_corrected)

    
    # With title, we can fetch associated metadata we wrote earlier to stor_metadata
    episode_number = str(stor_metadata.loc[video_name_corrected,"id"])
    episode_link = stor_metadata.loc[video_name_corrected,"link"] 
    img_url = stor_metadata.loc[video_name_corrected,"img"] 
    
    # Save the video thumbnail for later use the UI
    with open("../nextjs/public/%s.jpg"%str(episode_number), 'wb') as f:
        response = requests.get(img_url)
        f.write(response.content)
        #print(f)
        print(" ")
    
    # Save metadata
    episode_metadatas=[{"source":video_name + " " +episode_number,"id":episode_number,"link":episode_link,"title":video_name} for s in episode_splits]
    metadatas.append(episode_metadatas)

All you need to know about cat allergies & what you can do about them!
['All you need to know about cat allergies & what you can do about them!']
All you need to know about cat allergies & what you can do about them!
 
My Cat From Hell BIG NEWS - Live from the Cat Cave with Jackson Galaxy 5⧸23
['My Cat From Hell BIG NEWS - Live from the Cat Cave with Jackson Galaxy 5/23', 'GOOD MEWS ONLY - Live from the Cat Cave with Jackson Galaxy 6/13', 'Live from the Cat Cave with Jackson Galaxy 5/9']
My Cat From Hell BIG NEWS - Live from the Cat Cave with Jackson Galaxy 5/23
 
Tips on helping prevent Door-Dashing cats!
['Tips on helping prevent Door-Dashing cats!']
Tips on helping prevent Door-Dashing cats!
 
Can my Cats Get Along？ Cat-to-Cat Body Language basics & Introduction Tips
['Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips']
Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips
 
Biting the Hand That Loves You： Aggression or Affection？
['Biti

In [8]:
# Join the list of lists 
splits_all = []
for sublist in splits:
    splits_all.extend(sublist)
metadatas_all = []
for sublist in metadatas:
    metadatas_all.extend(sublist)

In [9]:
pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment="us-west4-gcp-free")

index_name = "cat-gpt"
embeddings = OpenAIEmbeddings()
p = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

#if batch_number == 2: 
    #p.delete(delete_all=True)

In [10]:
# Add data in chunk to avoid data ingest errors
chunk_size = 100
last_chunk = 0
num_chunks = math.ceil(len(splits_all) / chunk_size)

for i in range(last_chunk, num_chunks):
    # Set chunks to add
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, len(splits_all))
    
    # Extract the current chunks
    current_splits = splits_all[start_idx:end_idx]
    current_metadatas = metadatas_all[start_idx:end_idx]
    
    # Add the current chunk to the vector database
    p.add_texts(texts = current_splits, metadatas=current_metadatas)

Upserted vectors:   0%|          | 0/100 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/100 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
print('done')

done
